In [1]:
# Jupyter notebook Python code voor Jeugdzorgdata Zuid Holland Zuid
# Auteur Kees van den Tempel, AI-labs BV, Tricht
# Versie 1.0
# 26-4-2021

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import sklearn
import datetime
#%matplotlib inline

decl = pd.read_excel("data/DS_Jeugddata_Declaraties.xlsx")

print(decl.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 663685 entries, 0 to 663684
Data columns (total 42 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   AanbiederId                    663685 non-null  int64  
 1   DeclaratieId                   663685 non-null  int64  
 2   Gemeente                       663685 non-null  object 
 3   Zorgsoort                      663685 non-null  object 
 4   Team                           663685 non-null  object 
 5   ContractId                     663685 non-null  int64  
 6   ContractStatusId               663685 non-null  int64  
 7   ContractStatus                 663685 non-null  object 
 8   ClientId                       663685 non-null  int64  
 9   ContractStartdatum             663685 non-null  int64  
 10  ContractEinddatum              663685 non-null  int64  
 11  Stopdatum                      147404 non-null  float64
 12  ProductId                     

In [2]:
#### PRODUCTEN
# Koppel de stamtabel producten.xlsx met de declaratie dataset, om oude productcodes te vertalen naar nieuwe
product=pd.read_excel("data/Producten.xlsx")
print(product.head())
print("Matrix VOOR het toevoegen van producten tabel via inner join: " + str(decl.shape))

# maak een inner join tussen de twee tabellen
decl = pd.merge(left=decl, right=product, left_on='ProductId', right_on='product_id')
print("Matrix NA het toevoegen van producten tabel via inner join: " + str(decl.shape))

cnt1 = decl['ProductId'].nunique()
decl['nProductID'] = np.where(decl['product_id_actueel'].isna(), decl['ProductId'], decl['product_id_actueel'])
decl.drop(['product_id_actueel', 'ProductId', 'product_wet_onderdeel', 'product_id'], axis=1, inplace=True)
cnt2 = decl['nProductID'].nunique()

print("Aantal product kenmerken was " + str(cnt1) + " en is nu " + str(cnt2))

   product_id  product_id_actueel product_wet_onderdeel z_prod_cat_perceel
0         938                 NaN             jeugdhulp                PGB
1         939                 NaN             jeugdhulp                PGB
2         940                 NaN             jeugdhulp                PGB
3         941               951.0             jeugdhulp                PGB
4         942                 NaN             jeugdhulp                PGB
Matrix VOOR het toevoegen van producten tabel via inner join: (663685, 42)
Matrix NA het toevoegen van producten tabel via inner join: (663685, 46)
Aantal product kenmerken was 456 en is nu 148


In [3]:
#### AANBIEDERS
# Koppel de stamtabel Aanbieders.xlsx met de declaratie dataset, om oude aanbieders te vertalen naar nieuwe
aanbieder=pd.read_excel("data/Aanbieders.xlsx")
print(aanbieder.head())
print("Matrix VOOR het toevoegen van aanbieders tabel via inner join: " + str(decl.shape))

# maak een inner join tussen de twee tabellen
decl = pd.merge(left=decl, right=aanbieder, left_on='AanbiederId', right_on='AanbiederId', how='left')
print("Matrix NA het toevoegen van aanbieders tabel via inner join: " + str(decl.shape))

# Converteer de aanbiederID
cnt1 = decl['AanbiederId'].nunique()
decl['nAanbiederID'] = np.where(decl['AanbiederIdActueel'].isna(), decl['AanbiederId'], decl['AanbiederIdActueel'])
decl.drop(['AanbiederIdActueel', 'AanbiederId'], axis=1, inplace=True)
cnt2 = decl['nAanbiederID'].nunique()

print("Aantal aanbieders was " + str(cnt1) + " en is nu " + str(cnt2))

   AanbiederId  AanbiederIdActueel
0         1007                1007
1         1032                1867
2         1064                1064
3         1135                1810
4         1150                2277
Matrix VOOR het toevoegen van aanbieders tabel via inner join: (663685, 43)
Matrix NA het toevoegen van aanbieders tabel via inner join: (663685, 44)
Aantal aanbieders was 387 en is nu 356


In [4]:
# Converteer alle datum string naar datetime objecten zodat je er mee kunt rekenen

import sys
from datetime import datetime

# Definieer een functie voor het converteren van een datum string naar een datetime-object
def ConvertToDate(d):
    if d != d:
        return d
    else:
        d = str(int((d/1000000)+1))
    s = datetime.strptime(d, '%Y%m%d')
    return s

for feat in ['PeriodeStartdatum', 'PeriodeEinddatum']:
    # Testr of alle strings dezelfde waarde hebben
    # FirstValue = len(str(jeugd[feat].iloc[0]))
    # print(FirstValue)
    # assert (jeugd[feat].astype(str).str.len() == FirstValue).all()

    # Converteer de datumstrings naar datum-objecten (datetime - object)
    decl[feat] = decl[feat].apply(lambda x: ConvertToDate(x))
    print('Datum-strings van de kolom ' + feat + ' geconverteerd !')


Datum-strings van de kolom PeriodeStartdatum geconverteerd !
Datum-strings van de kolom PeriodeEinddatum geconverteerd !


In [5]:
decl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 663685 entries, 0 to 663684
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   DeclaratieId                   663685 non-null  int64         
 1   Gemeente                       663685 non-null  object        
 2   Zorgsoort                      663685 non-null  object        
 3   Team                           663685 non-null  object        
 4   ContractId                     663685 non-null  int64         
 5   ContractStatusId               663685 non-null  int64         
 6   ContractStatus                 663685 non-null  object        
 7   ClientId                       663685 non-null  int64         
 8   ContractStartdatum             663685 non-null  int64         
 9   ContractEinddatum              663685 non-null  int64         
 10  Stopdatum                      147404 non-null  float64       
 11  

In [6]:
# Detecteer in welke kolom er NaN waarden zitten en maak er 0 van
nan_cols = [i for i in decl.columns if decl[i].isnull().any()]
print("Columns with NaN: " + str(nan_cols))
for var in nan_cols: decl[var].fillna(0, inplace=True)
    
# Detecteer in welke kolom er INF waarden zitten en maak er 0 van
inf_cols = [i for i in decl.columns if decl[i].isin([np.inf, -np.inf]).all()]
print("Columns with Inf: " + str(inf_cols))
for var in inf_cols: decl[var].replace([np.inf, -np.inf], np.nan, inplace=True)
decl.dropna(subset=[inf_cols], inplace=True)

# Detecteer welke kolommen een object zijn
categorical = [i for i in decl.columns if decl[i].dtype=='O']
print("Columns which are object: " + str(categorical))
for var in categorical: decl[var].str.strip()
print("leading and trailing spaces deleted")


Columns with NaN: ['Stopdatum', 'VerrekendMetDeclaratieId', 'Budgettype', 'BudgettypeId']
Columns with Inf: []
Columns which are object: ['Gemeente', 'Zorgsoort', 'Team', 'ContractStatus', 'Levereenheid', 'LevereenheidOmschrijving', 'Betaalfrequentie', 'Betaalmethode', 'TypeContract', 'DeclaratieStatus', 'Budgettype', 'Bron', 'Regio', 'z_prod_cat_perceel']
leading and trailing spaces deleted


In [7]:
def ZoekenVerwijderfouteDatumWaarden(df, kolom):
    # Converteer de waardes in de kolom naar datetime, en bekijk voor welke rijen dit niet lukt:
    not_datetime = pd.to_datetime(df[kolom], errors='coerce').isna()

    print("Aantal rijen met juiste dtype (false) en met de verkeerde dtype (True)")
    print(not_datetime.groupby(not_datetime).count())

    print("\nDeze rijen hebben de verkeerde dtype en worden verwijderd")
    print(df[not_datetime==True][kolom])
    print(df.shape)
    df.drop(df[not_datetime==True].index, axis=0, inplace=True)
    print(df.shape)

    # De kolom van de dataframe worden gedwongen op datetime64 gezet
    return df.astype({kolom: np.datetime64})

decl = ZoekenVerwijderfouteDatumWaarden(decl, "PeriodeStartdatum")
decl = ZoekenVerwijderfouteDatumWaarden(decl, "PeriodeEinddatum")

print("\nFoute datum waarden verwijderd")

Aantal rijen met juiste dtype (false) en met de verkeerde dtype (True)
PeriodeStartdatum
False    663685
Name: PeriodeStartdatum, dtype: int64

Deze rijen hebben de verkeerde dtype en worden verwijderd
Series([], Name: PeriodeStartdatum, dtype: datetime64[ns])
(663685, 43)
(663685, 43)
Aantal rijen met juiste dtype (false) en met de verkeerde dtype (True)
PeriodeEinddatum
False    663685
Name: PeriodeEinddatum, dtype: int64

Deze rijen hebben de verkeerde dtype en worden verwijderd
Series([], Name: PeriodeEinddatum, dtype: datetime64[ns])
(663685, 43)
(663685, 43)

Foute datum waarden verwijderd


In [8]:
# declaratie die verekend zijn worden gewist. Het is onduidelijk hoe deze declaraties precies verekend zijn
# dus of het bedrag verlaagd is, hoeveelheid aangepast, etc

decl.drop(decl[decl["VerrekendMetDeclaratieId"]>0].index, axis=0, inplace=True)

In [9]:
# Bepaal of er binnen een contract meer dan 1 product gebruikt worden
# https://www.kite.com/python/answers/how-to-count-unique-values-in-a-pandas-dataframe-group-in-python

grouped_df = decl.groupby('ContractId')
grouped_df = grouped_df.agg({"nProductID": "nunique"})
grouped_df = grouped_df.reset_index()
print(grouped_df[grouped_df['nProductID']>1])

# Er zijn dus veel contracten waar meerdere producten op gedeclareerd worden !!!
# In totaal 15155 contracten met 2 tot wel 8 producten
# Dit klop helemaal niet met het voorzieningen bestand waar maar 1 product per contract wordt gespecificeerd
# Navragen bij de auteur van dit databestand

       ContractId  nProductID
1170        71322           2
2862        79957           2
2905        80269           2
2909        80292           2
2943        80695           2
...           ...         ...
87208      506411           2
87209      506418           2
87223      506800           2
87270      507221           2
87299      508886           2

[15132 rows x 2 columns]


In [10]:
# Bepaal of er binnen een contract meer dan 1 aanbieder is

grouped_df = decl.groupby('ContractId')
grouped_df = grouped_df.agg({"nAanbiederID": "nunique"})
grouped_df = grouped_df.reset_index()
print(grouped_df[grouped_df['nAanbiederID']>1])

# Er zijn geen contracten waar meerdere aanbieders op gedeclareerd hebben !!!

Empty DataFrame
Columns: [ContractId, nAanbiederID]
Index: []


In [11]:
# LET OP: zijn deze gegevens alleen relevant als de contracten ook daadwerkelijk afgelopen zijn ????
# JA, want als een contract net gestart is, is de duur heel kort en dat klopt niet
# DUS niet de totale kosten per contract berekenen, DUS niet op contract-niveau, wel op Client-niveau
# BETER om features te berekenen op het tijdstip van de declaratie
# VOORBEELD om de voorgaande behandeltijd te berekenen

decl['__ContractDuur'] = decl['ContractEinddatum'] - decl['ContractStartdatum']

# Features bepalen
# decl["TotaleKostenClient"] = decl["GedeclareerdeTariefExclBtw"].groupby(decl['ClientId']).transform('sum')
decl['__MeanProductDecl'] = decl.groupby('nProductID')['GedeclareerdeKostenExclBtw'].transform("mean")
decl['__StdDevProductDecl'] = decl.groupby('nProductID')['GedeclareerdeKostenExclBtw'].transform("std")
decl['__AantalProductDecl'] = decl.groupby('nProductID')['GedeclareerdeKostenExclBtw'].transform("count")

decl['__MeanAanbiederDecl'] = decl.groupby('nAanbiederID')['GedeclareerdeKostenExclBtw'].transform("mean")
decl['__StdDevAanbiederDecl'] = decl.groupby('nAanbiederID')['GedeclareerdeKostenExclBtw'].transform("std")
decl['__AantalAanbiederDecl'] = decl.groupby('nAanbiederID')['GedeclareerdeKostenExclBtw'].transform("count")

decl['__MeanProductHvlh'] = decl.groupby(['nProductID', 'Levereenheid'])['Hoeveelheid'].transform("mean")
decl['__StdDevProductHvlh'] = decl.groupby(['nProductID', 'Levereenheid'])['Hoeveelheid'].transform("std")

decl['__MeanAanbiederHvlh'] = decl.groupby(['nAanbiederID', 'Levereenheid'])['Hoeveelheid'].transform("mean")
decl['__StdDevAanbiederHvlh'] = decl.groupby(['nAanbiederID', 'Levereenheid'])['Hoeveelheid'].transform("std")

decl["PeriodeStartdatum"] = pd.to_datetime(decl["PeriodeStartdatum"])
decl["PeriodeEinddatum"] = pd.to_datetime(decl["PeriodeEinddatum"])
decl["__DeclaratiePeriode"] = (decl["PeriodeEinddatum"]-decl["PeriodeStartdatum"]).dt.days

decl['__MeanProductDeclduur'] = decl.groupby('nProductID')['__DeclaratiePeriode'].transform("mean")
decl['__StdDevProductDeclduur'] = decl.groupby('nProductID')['__DeclaratiePeriode'].transform("std")

decl['__MeanAanbiederDeclduur'] = decl.groupby('nAanbiederID')['__DeclaratiePeriode'].transform("mean")
decl['__StdDevAanbiederDeclduur'] = decl.groupby('nAanbiederID')['__DeclaratiePeriode'].transform("std")

decl['__MeanClientDeclduur'] = decl.groupby('ClientId')['__DeclaratiePeriode'].transform("mean")
decl['__StdDevClientDeclduur'] = decl.groupby('ClientId')['__DeclaratiePeriode'].transform("std")

# Bereken uit de start- en PeriodeStartdatum de Declaratieduur
# Alleen relevant als de contracten afgelopen zijn
if 1==2:
    decl['__TotaleKostenperContract'] = decl.groupby('ContractId')['GedeclareerdeKostenExclBtw'].transform("sum")

    decl['__DeclstartDatumContract'] = decl.groupby('ContractId')["PeriodeStartdatum"].transform('first')
    decl['__DecleindDatumContract'] = decl.groupby('ContractId')["PeriodeEinddatum"].transform('last')
    decl['__DeclduurContract'] = (decl['__DeclaratieeindDatum'] - decl['__DeclaratiestartDatum']).dt.days

    decl['__DeclstartDatumTotaal'] = decl.groupby('ClientId')['PeriodeStartdatum'].transform("first")
    decl['__DecleindDatumTotaal'] = decl.groupby('ClientId')['PeriodeEinddatum'].transform("last")
    decl['__DeclduurTotNuToe'] = (decl['__DecleindDatumTotaal'] - decl['__DeclstartDatumTotaal']).dt.days

print('Heel veel features berekend !\n')

print(decl.info())
decl.head()

Heel veel features berekend !

<class 'pandas.core.frame.DataFrame'>
Int64Index: 615867 entries, 0 to 663684
Data columns (total 61 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   DeclaratieId                   615867 non-null  int64         
 1   Gemeente                       615867 non-null  object        
 2   Zorgsoort                      615867 non-null  object        
 3   Team                           615867 non-null  object        
 4   ContractId                     615867 non-null  int64         
 5   ContractStatusId               615867 non-null  int64         
 6   ContractStatus                 615867 non-null  object        
 7   ClientId                       615867 non-null  int64         
 8   ContractStartdatum             615867 non-null  int64         
 9   ContractEinddatum              615867 non-null  int64         
 10  Stopdatum                      615867

,DeclaratieId,Gemeente,Zorgsoort,Team,ContractId,ContractStatusId,ContractStatus,ClientId,ContractStartdatum,ContractEinddatum,...,__StdDevProductHvlh,__MeanAanbiederHvlh,__StdDevAanbiederHvlh,__DeclaratiePeriode,__MeanProductDeclduur,__StdDevProductDeclduur,__MeanAanbiederDeclduur,__StdDevAanbiederDeclduur,__MeanClientDeclduur,__StdDevClientDeclduur
0,10181,Dordrecht,Jeugdzorg,Jeugdteam Dordrecht West,27316,5800,Afgelopen,17674,20150200000000,20160200000000,...,3403.024986,998.506212,3247.206119,28,11.970307,14.99525,10.929070,18.930032,16.402299,15.232661
1,10184,Papendrecht,Jeugdzorg,Jeugdteam Papendrecht,42238,5800,Afgelopen,15024,20150100000000,20150800000000,...,3403.024986,998.506212,3247.206119,28,11.970307,14.99525,10.929070,18.930032,25.785714,10.969739
2,10185,Papendrecht,Jeugdzorg,Jeugdteam Papendrecht,42239,5800,Afgelopen,14706,20150100000000,20150800000000,...,3403.024986,998.506212,3247.206119,28,11.970307,14.99525,10.929070,18.930032,48.333333,88.234806
3,14479,Dordrecht,Jeugdzorg,Jeugdteam Dordrecht West,16832,5800,Afgelopen,14204,20150200000000,20160200000000,...,3403.024986,175.028571,53.418154,28,11.970307,14.99525,29.485714,11.682998,48.400000,80.228068
4,14480,Dordrecht,Jeugdzorg,Jeugdteam Dordrecht West,16832,5800,Afgelopen,14204,20150200000000,20160200000000,...,3403.024986,175.028571,53.418154,31,11.970307,14.99525,29.485714,11.682998,48.400000,80.228068


In [13]:
# FEATURE ENGINEERING
# Bepaal drie nieuwe velden, die bekend zijn op het moment dat de beschikking wordt afgegeven:
# 1. EerdereKosten bij afgifte beschikking
# 2. EerdereDeclaratieduur bij afgifte beschikking
# 3. Volgnummer bij afgifte beschikking
# 4. Contractperiode
# Deze nieuwe kenmerken geven een beeld van de voorgeschiedenis van een client, en kunnen mogelijk een correlatie hebben
# met de hoogte van de nieuw af te geven beschikking. Later zal blijken of deze velden van enige importantie zijn

decl.sort_values(['PeriodeStartdatum'], inplace=True, ascending=True)

decl['__EerdereDeclaraties'] = decl.groupby(['ClientId'])['GedeclareerdeKostenExclBtw'].cumsum()
decl['__Volgnummer'] = decl.groupby(['ClientId'])['ClientId'].cumcount()

def EerdereDeclaratieduur(firstdate, mdate):
    Declaratieduur = (mdate-firstdate).days
    return Declaratieduur

decl['__EersteDeclaratieDatum'] = decl.groupby(['ClientId'])['PeriodeStartdatum'].transform('first')
decl["__EerdereDeclaratieduur"] = decl[['__EersteDeclaratieDatum', 'PeriodeStartdatum']].apply(lambda x: EerdereDeclaratieduur(*x), axis=1)

decl[decl['ClientId']==14204][['ClientId', '__Volgnummer', '__EerdereDeclaraties', 'GedeclareerdeKostenExclBtw', '__EersteDeclaratieDatum', 'PeriodeStartdatum', '__EerdereDeclaratieduur']].head(12)

,ClientId,__Volgnummer,__EerdereDeclaraties,GedeclareerdeKostenExclBtw,__EersteDeclaratieDatum,PeriodeStartdatum,__EerdereDeclaratieduur
4,14204,0,257.16,257.16,2015-01-01,2015-01-01,0
79557,14204,1,4011.35,3754.19,2015-01-01,2015-01-01,0
68,14204,2,4825.69,814.34,2015-01-01,2015-02-01,31
3,14204,3,5104.28,278.59,2015-01-01,2015-02-01,31
6,14204,4,5447.16,342.88,2015-01-01,2015-04-01,90
8,14204,5,5790.04,342.88,2015-01-01,2015-04-01,90
10,14204,6,6432.94,642.90,2015-01-01,2015-05-01,120
40,14204,7,7032.98,600.04,2015-01-01,2015-06-01,151
67,14204,8,8044.48,1011.50,2015-01-01,2015-07-01,181
69,14204,9,8387.36,342.88,2015-01-01,2015-08-01,212


In [14]:
decl['__KalenderMaand'] = pd.DatetimeIndex(decl['PeriodeStartdatum']).month
# decl['DeclSinus'] = decl['DeclaratieMaand'].apply(lambda x: np.sin(2 * np.pi * float(x)/365))



In [15]:
# Detecteer in welke kolom er NaN waarden zitten en maak er 0 van
nan_cols = [i for i in decl.columns if decl[i].isnull().any()]
print("Columns with NaN: " + str(nan_cols))
for var in nan_cols: decl[var].fillna(0, inplace=True)
    
# Detecteer in welke kolom er INF waarden zitten en maak er 0 van
inf_cols = [i for i in decl.columns if decl[i].isin([np.inf, -np.inf]).all()]
print("Columns with Inf: " + str(inf_cols))
for var in inf_cols: decl[var].replace([np.inf, -np.inf], np.nan, inplace=True)
decl.dropna(subset=[inf_cols], inplace=True)


Columns with NaN: ['__StdDevProductDecl', '__StdDevAanbiederDecl', '__StdDevProductHvlh', '__StdDevAanbiederHvlh', '__StdDevProductDeclduur', '__StdDevAanbiederDeclduur', '__StdDevClientDeclduur']
Columns with Inf: []


In [16]:
decl.drop(['VerrekendMetDeclaratieId', '__EersteDeclaratieDatum', 'Gemeente', 'Zorgsoort', 'Team', 'ContractStatus', 'Stopdatum', 'LevereenheidOmschrijving', 'DeclaratieStatus', 'Bron', 'Regio', 'z_prod_cat_perceel', 'PeriodeStartdatum', 'PeriodeEinddatum'], axis=1, inplace=True)
decl.to_csv("data/Declaraties_autoencoder.csv")